# Create virtual machines in Orbstack(MacOS)/ multipass(Ubuntu)

This setup is meant for running Ansible playbooks on MacOS/Ubuntu. 

## Prerequisites for MacOs
1. Install [OrbStack](orbstack.dev)
2. Confirm `orb` command is now available on your command line
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/intro_installation.html)

## Prerequisites for Ubuntu
1. Install [multipass](https://canonical.com/multipass/install)
2. Confirm installation with `multipass --version` command
3. Install [Ansible](https://docs.ansible.com/ansible/latest/installation_guide/installation_distros.html#installing-ansible-on-debian)

Either run the commands manually from command like (for live output) or by executing cells here

Identifying the OS you are currently using

In [1]:
import platform

linux = platform.system() == "Linux"

Running the provision script according to OS

In [3]:
import subprocess
if linux:
    subprocess.run(["bash", "./provision-linux.sh"], check=True)
else:
    subprocess.run(["bash", "./provision-macos.sh"], check=True)


Removing existing VM: manager
Removing existing VM: worker
Provisioning user on manager...
info: Adding group `provision' (GID 1000) ...
info: Adding user `provision' ...
info: Adding new user `provision' (1000) with group `provision (1000)' ...
info: Creating home directory `/home/provision' ...
info: Copying files from `/etc/skel' ...
info: Adding new user `provision' to supplemental / extra groups `users' ...
info: Adding user `provision' to group `users' ...
provision ALL=(ALL) NOPASSWD:ALL
Provisioning user on worker...
info: Adding group `provision' (GID 1000) ...
info: Adding user `provision' ...
info: Adding new user `provision' (1000) with group `provision (1000)' ...
info: Creating home directory `/home/provision' ...
info: Copying files from `/etc/skel' ...
info: Adding new user `provision' to supplemental / extra groups `users' ...
info: Adding user `provision' to group `users' ...
provision ALL=(ALL) NOPASSWD:ALL
Generating SSH key on manager...
Generating public/private r

Notice that the following task will prompt (in VSCode) for your dockerhub username and password

In [4]:
import os

os.environ["DOCKER_USERNAME"] = input("Enter Dockerhub username: ")
os.environ["DOCKER_PASSWORD"] = input("Enter Dockerhub password: ")

The next command is run with Python instead of %%bash simply because %%bash doesn't support real-time output

In [5]:
inventory = 'local.macos.yml'
if linux:
    inventory = 'local.linux.yml'

cmd = f"""
mongo_user=$(openssl rand -hex 8)
mongo_pass=$(openssl rand -base64 24)
backup_pass=$(openssl rand -base64 16)
es_pass=$(openssl rand -base64 24)

ansible-playbook -i {inventory} ../server-setup/playbook.yml \\
  -e encrypted_disk_size=200g \\
  -e dockerhub_username=$DOCKER_USERNAME \\
  -e dockerhub_password=$DOCKER_PASSWORD \\
  -e mongodb_admin_username=$mongo_user \\
  -e mongodb_admin_password="$mongo_pass" \\
  -e backup_encryption_passphrase="$backup_pass" \\
  -e elasticsearch_superuser_password="$es_pass" \\
  -vv
"""

In [6]:
import subprocess

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True, env=os.environ)
for line in proc.stdout:
    print(line, end="")
proc.wait()

ansible-playbook [core 2.18.6]
  config file = None
  configured module search path = ['/Users/vmudryi/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /opt/homebrew/Cellar/ansible/11.6.0/libexec/lib/python3.13/site-packages/ansible
  ansible collection location = /Users/vmudryi/.ansible/collections:/usr/share/ansible/collections
  executable location = /opt/homebrew/bin/ansible-playbook
  python version = 3.13.3 (main, Apr  8 2025, 13:54:08) [Clang 17.0.0 (clang-1700.0.13.3)] (/opt/homebrew/Cellar/ansible/11.6.0/libexec/bin/python)
  jinja version = 3.1.6
  libyaml = True
No config file found; using defaults
[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details
redirecting (type: modules) ansible.builtin.ufw to community.general.ufw
Skipping callback 'default', as we already have a stdout callback.
Skipping callback 'minimal', as we already have a stdout callback.
Skipping callback 'oneline

4